In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv

load_dotenv() 

True

In [3]:
import sys, os
import openai

from langgraph.graph import StateGraph, END

In [4]:
from data.loaders import load_businesses, load_reviews
from data.builders import build_documents

from agents import *
from retrieval import *

In [5]:
# OpenAI Setup
openai_client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [6]:
# DOCUMENTS
restaurants = load_businesses()
reviews_by_business = load_reviews()
docs = build_documents(restaurants, reviews_by_business)

docs_ids = [d["id"] for d in docs]

len(docs), len(docs_ids)

Loading businesses...
Restaurants found: 52268
Loading reviews...
Total restaurants with reviews: 150346
Building documents...
Total documents: 52268


(52268, 52268)

In [7]:
# EMBEDDINGS
embedder = DocumentEmbedder()
embeddings = embedder.encode_docs(docs)

embeddings.shape

Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
Embedding documents...


Batches:   0%|          | 0/817 [00:00<?, ?it/s]

(52268, 384)

In [8]:
# FAISS
embedding_dim = embeddings.shape[1]
faiss_index = FaissIndex(embedding_dim)
faiss_index.add(embeddings, docs, docs_ids)

print("Index size:", faiss_index.size())

Index size: 52268


In [9]:
# RERANKER (Cross-encoder)
reranker = Reranker()

In [10]:
# AGENTS 
cuisine_agent = CuisineAgent(openai_client)
location_agent = LocationAgent(openai_client)
recommendation_agent = RecommendationAgent(embedder, faiss_index, reranker, openai_client)

In [11]:
# LangGraph
graph = StateGraph(RestaurantRecommendationState)

graph.add_node("cuisine", cuisine_agent.run)
graph.add_node("location", location_agent.run)
graph.add_node("recommendation", recommendation_agent.run)

graph.set_entry_point("cuisine")
graph.add_edge("cuisine", "location")
graph.add_edge("location", "recommendation")
graph.add_edge("recommendation", END)

app = graph.compile()

In [12]:
def get_restaurant_recommendations(query: str):
    return app.invoke({"user_query": query})

In [13]:
# TEST
response = get_restaurant_recommendations("Suggest restaurants near downtown Nashville with vegetarian options, moderately priced, suitable for a family dinner.")
print(response['recommendations'])

**Calypso Cafe**
- Match reason: Calypso Cafe offers a variety of vegetarian options, including a vegan version of their Black Bean Salad and a Vegetarian Family Carryout Special. The restaurant is praised for its affordable prices and family-friendly atmosphere, making it a great choice for a family dinner.
- Score: 9/10

**The Southern V**
- Match reason: Known for its excellent variety of vegetarian and vegan Southern cuisine, The Southern V provides a unique dining experience with dishes like vegan "chicken" biscuits. The friendly service and fair pricing make it a fantastic option for a family dinner.
- Score: 8.5/10

**Graze Nashville**
- Match reason: Graze Nashville specializes in vegetarian and vegan cuisine, offering a diverse menu with fresh and flavorful options. The restaurant's welcoming atmosphere and affordable brunch menu make it a suitable choice for a family outing.
- Score: 8/10
